In [ ]:
from utils import create_model
model = create_model()
model.load_weights("results/model.h5")

In [ ]:
import os
import tensorflow as tf
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

def classify_gender(file_path):
    # Extract features and reshape them
    features = extract_feature(file_path, mel=True).reshape(1, -1)
    # Predict the gender
    male_prob = model.predict(features, verbose=0)[0][0]
    female_prob = 1 - male_prob
    gender = "male" if male_prob > female_prob else "female"
    return gender

def process_file(emotion_folder_path, emotion_output_dir, file, progress_bar):
    if file.endswith(".wav"):
        file_path = os.path.join(emotion_folder_path, file)
        # Classify gender of the file
        classification = classify_gender(file_path)
        # Save only if classified as female
        if classification == "female":
            output_file_path = os.path.join(emotion_output_dir, file)
            os.replace(file_path, output_file_path)  # Move or copy the file to the output directory
        # Update the progress bar
        progress_bar.update(1)

def process_emotion_folder(emotion_folder, dataset_path, output_dir):
    emotion_folder_path = os.path.join(dataset_path, emotion_folder)
    if os.path.isdir(emotion_folder_path):
        # Creating a sub-directory for the emotion in the filtered dataset
        emotion_output_dir = os.path.join(output_dir, emotion_folder)
        os.makedirs(emotion_output_dir, exist_ok=True)

        # Iterating through each WAV file in the emotion folder
        files = [f for f in os.listdir(emotion_folder_path) if f.endswith(".wav")]
        with tqdm(total=len(files), desc=f'Processing {emotion_folder}') as progress_bar:
            with ThreadPoolExecutor() as executor:
                futures = [executor.submit(process_file, emotion_folder_path, emotion_output_dir, file, progress_bar) for file in files]
                for future in futures:
                    future.result()  
        progress_bar.update(1)

def main(dataset_path, output_dir):
    # Creating output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterating through each emotion folder
    emotion_folders = [f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))]
    for emotion_folder in emotion_folders:
        process_emotion_folder(emotion_folder, dataset_path, output_dir)

    print("Female voice extraction completed.")

In [ ]:
dataset_path = r'D:\NullClass_Internship\Audio_Emotion_Prediction\Emotions'
output_dir = r'D:\NullClass_Internship\Audio_Emotion_Prediction\filtered_dataset'

In [ ]:
main(dataset_path, output_dir)